## QC: Urbansim matches Estimates for 2017

###  Urbansim 2017 units should match 2017 housing unit by Jurisdiction

In [ ]:
import os
import sys

In [ ]:
# append path to find utils module in urbansim
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
parentdir2 =  os.path.abspath(os.path.join(parentdir, os.pardir))
sys.path.append(parentdir2) # to get path to utils module

In [ ]:
import pandas as pd
import utils

In [ ]:
from sqlalchemy import create_engine
from database import get_connection_string

In [ ]:
%matplotlib inline

In [ ]:
# connect to database
db_connection_string = get_connection_string('..\..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
# get versions
versions = utils.yaml_to_dict('../../data/scenario_config.yaml', 'scenario')

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\n   Max run id : {:,}".format(run_id))

# input run id (optional)

In [ ]:
#run_id = input()

In [ ]:
#print(run_id)

# get estimates and urbansim output

In [ ]:
output_sql = '''
SELECT jurisdiction.name as City,  
           parcel2015_mgra_jur_cpa.jur_id,
           sum(unit_change) as urbansim_unit_change
    FROM urbansim.urbansim.urbansim_lite_output urbansim_lite_output 
    JOIN [isam].[xpef04].[parcel2015_mgra_jur_cpa] parcel2015_mgra_jur_cpa 
        ON parcel2015_mgra_jur_cpa.parcel_id = urbansim_lite_output.parcel_id
    JOIN urbansim.ref.jurisdiction jurisdiction
        ON parcel2015_mgra_jur_cpa.jur_id = jurisdiction.jurisdiction_id
    WHERE  i = 1 and run_id =  %s and year_simulation = 2017
    GROUP BY jurisdiction.name,parcel2015_mgra_jur_cpa.jur_id
    ORDER BY jurisdiction.name,parcel2015_mgra_jur_cpa.jur_id'''
output_sql = output_sql % run_id
output = pd.read_sql(output_sql,mssql_engine)

In [ ]:
targets2017_sql = '''
SELECT [jurisdiction_id] as jur_id
      ,[housing_units_add] AS estimates_target_units
  FROM [urbansim].[urbansim].[urbansim_target_hu_jur]
  WHERE [version_id] = %s
  ORDER BY [yr], [jurisdiction_id]
'''
targets2017_sql = targets2017_sql % versions['subregional_targets_id']
targets2017 = pd.read_sql(targets2017_sql, mssql_engine)

In [ ]:
jur2017 = pd.merge(targets2017,output,on='jur_id')

In [ ]:
jur2017['jur_id'] = jur2017['jur_id'].astype('int')

In [ ]:
jur2017['diff'] =jur2017['estimates_target_units'] - jur2017['urbansim_unit_change']

In [ ]:
jur2017.set_index('jur_id',inplace=True)
del jur2017.index.name

In [ ]:
jur2017

# diagnostic plot

In [ ]:
ax = jur2017.plot(xticks=jur2017.index,rot=90,style='.-',subplots=True,figsize=(6, 6))
ax[0].set_xticklabels(jur2017["City"]);
ax[1].set_ylabel('housing units')
ptitle = 'DRAFT Estimates by City \n(version_id=' + \
str(versions['subregional_targets_id']) + ', run_id=' + str(run_id) + ')'
ax.flat[0].set_title(ptitle,size=16)

### tabular results of QC

In [ ]:
jur2017

# QC Pass/Fail

In [ ]:
unit_diff = jur2017['diff'].sum()
if unit_diff !=0:
    print('\n\nQC Fail: Urbansim units 2017 by City does not matches estimates\n\n')
else:
    print('\nUrbansim output for 2017 matches estimates\n\n')
    print('QC Pass\n\n\n')